In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用KFP SDK进行指标可视化和运行比较

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/metrics_viz_run_compare_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/metrics_viz_run_compare_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/metrics_viz_run_compare_kfp.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
     </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本笔记本演示如何使用[Kubeflow Pipelines（KFP）SDK](https://www.kubeflow.org/docs/components/pipelines/)构建[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)，生成模型指标和指标可视化，并比较管道运行情况。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

### 目标

在本教程中，您将学习如何使用KFP SDK构建生成评估指标的管道。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Pipelines`

执行的步骤包括：

- 创建KFP组件：
    - 为分类结果生成ROC曲线和混淆矩阵可视化
    - 编写指标
- 创建KFP管道。
- 执行KFP管道
- 比较管道运行中的指标

数据集

本教程使用的数据集是[葡萄酒数据集](https://archive.ics.uci.edu/ml/datasets/wine)来自[Scikit-learn内置数据集](https://scikit-learn.org/stable/datasets.html)。

该数据集用于预测葡萄酒的产地。

该教程使用的数据集是来自Scikit-learn内置数据集的鸢尾花数据集。

该数据集预测了三种鸢尾花的类别：山鸢尾（setosa）、维吉尼亚鸢尾（virginica）和变色鸢尾（versicolor）。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 基于您的预测使用量生成成本估算。

安装

安装执行此笔记本所需的软件包。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade matplotlib $USER_FLAG -q

### 仅限Colab：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

##开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，用于支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API]。

4. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 “REGION” 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能会使用共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google云帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经完成了身份验证。

**2. 本地JupyterLab实例，取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何授予您的服务帐户Cloud Storage权限，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

- *{给笔记本作者的提示: 对于任何需要是唯一的用户提供的字符串（如存储桶名称或模型 ID），请在末尾添加“-unique”，以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令通过执行下面的第二个单元格来获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账户访问权限

运行以下命令为您的服务账户授予对在上一步创建的存储桶中的管道工件进行读取和写入的访问权限 — 您只需要针对每个服务账户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aip
from kfp import compiler, dsl
from kfp.dsl import ClassificationMetrics, Metrics, Output, component

设置以下常量以用于Vertex AI Pipelines:

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/iris".format(BUCKET_URI)

在此项目和对应的存储桶中初始化用于Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 使用scikit-learn定义流水线组件

在这个部分，您定义了一些基于Python函数的组件，这些组件使用scikit-learn来训练一些分类器，并生成可视化的评估结果。

请注意下面定义中使用的`@component()`装饰器。您可以选择设置组件安装的包列表；使用的基础镜像（默认为Python 3.7镜像）；以及生成组件YAML文件的名称，从而可以共享和重复使用组件定义。

#### 定义wine_classification组件

第一个组件展示了如何可视化*ROC曲线*。
请注意函数定义中包含一个名为`wmetrics`的输出，类型为`Output[ClassificationMetrics]`。您可以在Cloud控制台的Pipelines用户界面中可视化这些指标。

为了实现这一点，此示例使用了artifact的`log_roc_curve()`方法。该方法以假阳率、真阳率和阈值的数组作为输入，这些数组由[`sklearn.metrics.roc_curve`函数生成](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)。

当您评估下面的单元格时，将创建一个名为`wine_classification`的任务工厂函数，用于构建流水线定义。此外，将创建一个组件YAML文件，可以通过文件或URL进行共享和加载，以创建相同的任务工厂函数。

In [ ]:
@component(packages_to_install=["scikit-learn==1.2.2"], base_image="python:3.9")
def wine_classification(wmetrics: Output[ClassificationMetrics]):
    from sklearn.datasets import load_wine
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import cross_val_predict, train_test_split

    X, y = load_wine(return_X_y=True)
    # Binary classification problem for label 1.
    y = y == 1

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    rfc = RandomForestClassifier(n_estimators=10, random_state=42)
    rfc.fit(X_train, y_train)
    y_scores = cross_val_predict(rfc, X_train, y_train, cv=3, method="predict_proba")
    fpr, tpr, thresholds = roc_curve(
        y_true=y_train, y_score=y_scores[:, 1], pos_label=True
    )
    wmetrics.log_roc_curve(fpr, tpr, thresholds)


compiler.Compiler().compile(wine_classification, "wine_classification_component.yaml")

#### 定义iris_sgdclassifier组件

第二个组件展示了如何可视化一个*混淆矩阵*，在这种情况下，是针对使用`SGDClassifier`训练的模型。

与之前的组件一样，您创建一个类型为`Output[ClassificationMetrics]`的`metricsc`输出工件。 然后，使用工件的`log_confusion_matrix`方法来可视化混淆矩阵结果，这是由[sklearn.metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)函数生成的。

In [ ]:
@component(packages_to_install=["scikit-learn==1.2.2"], base_image="python:3.9")
def iris_sgdclassifier(
    test_samples_fraction: float,
    metricsc: Output[ClassificationMetrics],
):
    from sklearn import datasets, model_selection
    from sklearn.linear_model import SGDClassifier
    from sklearn.metrics import confusion_matrix

    iris_dataset = datasets.load_iris()
    train_x, test_x, train_y, test_y = model_selection.train_test_split(
        iris_dataset["data"],
        iris_dataset["target"],
        test_size=test_samples_fraction,
    )

    classifier = SGDClassifier()
    classifier.fit(train_x, train_y)
    predictions = model_selection.cross_val_predict(classifier, train_x, train_y, cv=3)
    metricsc.log_confusion_matrix(
        ["Setosa", "Versicolour", "Virginica"],
        confusion_matrix(
            train_y, predictions
        ).tolist(),  # .tolist() to convert np array to list.
    )

#### 定义 iris_logregression 组件

第三个组件也使用“iris”数据集，但训练一个 `LogisticRegression` 模型。 它在 `metrics` 输出 artifact 中记录模型的 `accuracy`。

In [ ]:
@component(
    packages_to_install=["scikit-learn==1.2.2"],
    base_image="python:3.9",
)
def iris_logregression(
    input_seed: int,
    split_count: int,
    metrics: Output[Metrics],
):
    from sklearn import datasets, model_selection
    from sklearn.linear_model import LogisticRegression

    # Load digits dataset
    iris = datasets.load_iris()
    # # Create feature matrix
    X = iris.data
    # Create target vector
    y = iris.target
    # test size
    test_size = 0.20

    # cross-validation settings
    kfold = model_selection.KFold(
        n_splits=split_count, random_state=input_seed, shuffle=True
    )
    # Model instance
    model = LogisticRegression()
    scoring = "accuracy"
    results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    print(f"results: {results}")

    # split data
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=input_seed
    )
    # fit model
    model.fit(X_train, y_train)

    # accuracy on test set
    result = model.score(X_test, y_test)
    print(f"result: {result}")
    metrics.log_metric("accuracy", (result * 100.0))

### 定义流水线

接下来，定义一个简单的流水线，使用在前一节中创建的组件。

In [ ]:
PIPELINE_NAME = "metrics-pipeline-v2"


@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="metrics-pipeline-v2",
)
def pipeline(seed: int, splits: int):
    wine_classification_op = wine_classification()  # noqa: F841
    iris_logregression_op = iris_logregression(  # noqa: F841
        input_seed=seed, split_count=splits
    )
    iris_sgdclassifier_op = iris_sgdclassifier(test_samples_fraction=0.3)  # noqa: F841

## 编译管道

接下来，编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular_classification_pipeline.yaml",
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "iris_" + UUID

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular_classification_pipeline.yaml",
    job_id=f"tabular-classification-v2{UUID}-1",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"seed": 7, "splits": 10},
)

job.run()

点击生成的链接查看您在云控制台中的运行情况。

<!-- 在运行时应该看起来像这样：

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" width="40%"/></a> -->

在用户界面中，当您点击它们时，许多管道DAG节点会展开或折叠。

## 在用户界面中比较管道运行

接下来，生成另一个使用不同的`seed`和`split`来运行`iris_logregression`步骤的管道运行。

提交新的管道运行：


**pipeline_root：** 指定一个 Cloud Storage URI，您的管道服务帐户可以访问。您的管道运行的工件存储在管道根目录中。

**display_name：** 管道的名称，这将显示在 Google Cloud 控制台中。

**parameter_values：** 传递给此运行的管道参数。例如，创建一个 dict()，其中参数名称为字典键，参数值为字典值。

**job_id：** 此管道运行的唯一标识符。如果未指定作业 ID，则 Vertex AI Pipelines 会使用管道名称和管道运行开始时间戳为您创建作业 ID。

**template_path：** 完整的管道路径

In [ ]:
job = aip.PipelineJob(
    display_name="iris_" + UUID,
    template_path="tabular_classification_pipeline.yaml",
    job_id=f"tabular-classification-pipeline-v2{UUID}-2",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"seed": 5, "splits": 7},
)

job.run()

当两个流水线运行都完成后，通过在云控制台中导航到流水线运行列表，选择它们两个，并在控制台面板顶部点击**比较**来比较它们的结果。

比较从其跟踪的元数据中运行的管道的参数和指标

接下来，您可以使用Python的Vertex AI SDK来比较管道运行的参数和指标。等到管道运行完成后再运行下一个单元。

In [ ]:
pipeline_df = aip.get_pipeline_df(pipeline=PIPELINE_NAME)
print(pipeline_df.head(2))

### 绘制参数和度量的平行坐标

通过在数据框中拥有度量和参数，您可以进行进一步的分析来提取有用的信息。下面的例子使用平行坐标图来比较每次运行的数据。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["figure.figsize"] = [15, 5]

pipeline_df["param.input:seed"] = pipeline_df["param.input:seed"].astype(np.float16)
pipeline_df["param.input:splits"] = pipeline_df["param.input:splits"].astype(np.float16)

ax = pd.plotting.parallel_coordinates(
    pipeline_df.reset_index(level=0),
    "run_name",
    cols=["param.input:seed", "param.input:splits", "metric.accuracy"],
)
ax.set_yscale("symlog")
ax.legend(bbox_to_anchor=(1.0, 0.5))

绘制ROC曲线并计算AUC数值

除了基本指标外，您还可以使用`get_pipeline_df`方法提取复杂指标并进行进一步分析。

In [ ]:
try:
    df = pd.DataFrame(pipeline_df["metric.confidenceMetrics"][0])
    auc = np.trapz(df["recall"], df["falsePositiveRate"])
    plt.plot(df["falsePositiveRate"], df["recall"], label="auc=" + str(auc))
    plt.legend(loc=4)
    plt.show()
except Exception as e:
    print(e)

# 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源 -- *注意:* 这是自动生成的，不是所有资源都适用于此教程。

In [ ]:
import os

delete_bucket = False

job.delete()
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

! rm -rf tabular_classification_pipeline.yaml